In [15]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [16]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# IMPORTANTE TESTAR COM O ARQUIVO DE PREVISÃO GERADO NA DIFICULDADE DIFÍCIL, PARA CONSEGUIR TESTAR A MANIPULAÇÃO NO MODO FÁCIL E NORMAL
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

dificuldades = ['normal', 'facil']
criptos = ['Bitcoin', 'Ethereum', 'BNB', 'Solana']

acuracia = []

# Função para diminuir a acertividade do bot no modo facil e normal
def forcar_erros(df, taxa_erro_desejada):
    # Calcular número de previsões a serem alteradas
    n_modificar = int(len(df) * taxa_erro_desejada)
    
    # Selecionar índices aleatórios das previsões a modificar
    indices_modificar = np.random.choice(df.index, size=n_modificar, replace=False)
    
    # Inverter o sinal das previsões selecionadas
    df.loc[indices_modificar, 'Variação Prevista (%)'] *= -1
    
    return df

for dificuldade in dificuldades:
    for cripto in criptos:
        i=1
        while i<=30:
            
            base_diretório = Path(os.getcwd())
            diretório_princapl = base_diretório.parent
            caminho = diretório_princapl / 'data' / f'previsão semanal - {cripto}.csv'

            # Ler arquivo da base
            acao = pd.read_csv(caminho)
            
            df_semana = acao
            
            # Aplicar erro conforme a dificuldade
            if dificuldade == 'normal':
                df_semana = forcar_erros(df_semana, 0.50)  # Força erro em 50% das previsões
            elif dificuldade == 'facil':
                df_semana = forcar_erros(df_semana, 0.75)  # Força erro em 75% das previsões
            
            # Calcular se a direção do valor previsto foi igual ao real (1 = acertou | 0 = errou)
            df_semana['direcao_correta'] = np.where(
                (df_semana['Variação Real (%)'] > 0) & (df_semana['Variação Prevista (%)'] > 0) |
                (df_semana['Variação Real (%)'] < 0) & (df_semana['Variação Prevista (%)'] < 0),
                1, 0
            )

            # Calcular a métrica de Erro de Direção (Directional Accuracy)
            directional_accuracy = df_semana['direcao_correta'].mean()

            # Exibir resultados
            print(f"{i}- Directional Accuracy da {cripto} no modo {dificuldade}: {directional_accuracy * 100:.2f}%")
            # acuracia.append(directional_accuracy)
            # Armazenando os resultados
            acuracia.append({
                'criptomoeda': cripto,
                'dificuldade': dificuldade,
                'directional accuracy (%)': directional_accuracy*100
            })
            i+=1

results_df = pd.DataFrame(acuracia)


1- Directional Accuracy da Bitcoin no modo normal: 48.00%
2- Directional Accuracy da Bitcoin no modo normal: 47.00%
3- Directional Accuracy da Bitcoin no modo normal: 48.00%
4- Directional Accuracy da Bitcoin no modo normal: 50.00%
5- Directional Accuracy da Bitcoin no modo normal: 50.00%
6- Directional Accuracy da Bitcoin no modo normal: 48.00%
7- Directional Accuracy da Bitcoin no modo normal: 53.00%
8- Directional Accuracy da Bitcoin no modo normal: 43.00%
9- Directional Accuracy da Bitcoin no modo normal: 45.00%
10- Directional Accuracy da Bitcoin no modo normal: 51.00%
11- Directional Accuracy da Bitcoin no modo normal: 54.00%
12- Directional Accuracy da Bitcoin no modo normal: 51.00%
13- Directional Accuracy da Bitcoin no modo normal: 48.00%
14- Directional Accuracy da Bitcoin no modo normal: 45.00%
15- Directional Accuracy da Bitcoin no modo normal: 51.00%
16- Directional Accuracy da Bitcoin no modo normal: 46.00%
17- Directional Accuracy da Bitcoin no modo normal: 52.00%
18- Di

In [17]:
results_df

,criptomoeda,dificuldade,directional accuracy (%)
0,Bitcoin,normal,48.0
1,Bitcoin,normal,47.0
2,Bitcoin,normal,48.0
3,Bitcoin,normal,50.0
4,Bitcoin,normal,50.0
...,...,...,...
235,Solana,facil,36.0
236,Solana,facil,28.0
237,Solana,facil,28.0
238,Solana,facil,32.0


In [18]:
results_df.to_excel("Resultados testes dificuldades/Resultado geral dificuldade.xlsx", engine="openpyxl", index=False)

# Agrupar pelas combinações de parâmetros e calcular a média das métricas
resultados_agrupados = results_df.groupby(['criptomoeda', 'dificuldade']).agg({
    'directional accuracy (%)': 'mean'
}).reset_index()

# Exibir os resultados agrupados
print(resultados_agrupados)

resultados_agrupados.to_excel("Resultados testes dificuldades/Média resultado geral dificuldade.xlsx", engine="openpyxl", index=False)

  criptomoeda dificuldade  directional accuracy (%)
0         BNB       facil                 31.500000
1         BNB      normal                 50.500000
2     Bitcoin       facil                 29.366667
3     Bitcoin      normal                 49.266667
4    Ethereum       facil                 30.966667
5    Ethereum      normal                 49.800000
6      Solana       facil                 31.566667
7      Solana      normal                 48.400000
